# Trying to remotely access DICOM tags, pixel arrays from DICOMs, and ROIs from DICOM-RTSTRUCT files (without downloading the files locally first)

In [1]:
import xnat
#xnat.__version__ # 0.3.21
print(f'XNATpy version = {xnat.__version__}') # 0.3.21
import SimpleITK as sitk
import pydicom
import dicom2nifti
import numpy as np
#import math
import matplotlib.pyplot as plt
#from matplotlib import pyplot
import matplotlib.image as mpimg
import os, sys, time
import natsort
import dicom_functions
import explore_dicom_data
from myshow import myshow
import importlib

XNATpy version = 0.3.21


# Connect to XNAT server:

In [2]:
xnatAddress = 'http://10.1.1.17'

session = xnat.connect(xnatAddress, user='admin', password='admin')
#session = xnat.connect(xnatAddress, user='admin', password='admin', debug=True)

# Define the REST path variables:

In [3]:
# Define selected project and subject labels, and session number:
projectLabel = 'BrainTumorProg'
subjectLabel = 'PGM-002'
experimentNo = 0 # (= 1 in XNAT web app; e.g. ‘PGM-002_MR_1’)
scanNo = 0 # 0 -> T1post

# Try to access DICOM tags and pixel arrays without downloading the files:

In [4]:
scans = session.projects[projectLabel].subjects[subjectLabel].experiments[experimentNo].scans[scanNo]

scans

<MrScanData T1post (11)>

In [5]:
# This reads only the header and not the pixel data:
# (see:
# https://xnat.readthedocs.io/en/latest/static/tutorial.html#accessing-xnat-files-as-local-files-partial-read)
# )
scans.read_dicom()

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'OTHER']
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.14519.5.2.1.4429.7055.261648268414748489539476953118
(0008, 0020) Study Date                          DA: '19960813'
(0008, 0021) Series Date                         DA: '19960813'
(0008, 0022) Acquisition Date                    DA: '19960813'
(0008, 0023) Content Date                        DA: '19960813'
(0008, 0030) Study Time                          TM: '164104.000000'
(0008, 0031) Series Time                         TM: '162908'
(0008, 0032) Acquisition Time                    TM: '162908'
(0008, 0033) Content Time                        TM: '162908'
(0008, 0050) Accession Number                    SH: '7351166753651521'
(0008, 0060) Modality                            CS: 'MR'
(0008, 0070) Manufa

In [6]:
# To read the pixel data:
scans.read_dicom(read_pixel_data=True)

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'OTHER']
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.14519.5.2.1.4429.7055.261648268414748489539476953118
(0008, 0020) Study Date                          DA: '19960813'
(0008, 0021) Series Date                         DA: '19960813'
(0008, 0022) Acquisition Date                    DA: '19960813'
(0008, 0023) Content Date                        DA: '19960813'
(0008, 0030) Study Time                          TM: '164104.000000'
(0008, 0031) Series Time                         TM: '162908'
(0008, 0032) Acquisition Time                    TM: '162908'
(0008, 0033) Content Time                        TM: '162908'
(0008, 0050) Accession Number                    SH: '7351166753651521'
(0008, 0060) Modality                            CS: 'MR'
(0008, 0070) Manufa

In [7]:
temp = scans.read_dicom(read_pixel_data=True).PixelData
len(temp)

524288

Looks like the pixel data is flat! 

Maybe pixel data is flattened like contour data is, so I'll try replacing "read_pixel_data=True" with
"read_pixel_array=True":

In [8]:
# Try this:
scans.read_dicom(read_pixel_array=True)

TypeError: read_dicom() got an unexpected keyword argument 'read_pixel_array'

Maybe the clue is here:

https://xnat.readthedocs.io/en/latest/static/tutorial.html#exploring-your-xnat-server

under "Accessing XNAT files as local files (partial read)"

In [9]:
session.projects[projectLabel].subjects[subjectLabel].experiments[experimentNo].scans[scanNo]

<MrScanData T1post (11)>

In [10]:
session.projects[projectLabel].subjects[subjectLabel].experiments[experimentNo].scans[scanNo].resources['DICOM']

<ResourceCatalog DICOM (41)>

In [11]:
session.projects[projectLabel].subjects[subjectLabel].experiments[experimentNo].scans[scanNo].resources['DICOM'].files

<XNATListing {(1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313-11-19-abhab5.dcm, 1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313-11-19-abhab5.dcm): <FileData 1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313-11-19-abhab5.dcm (1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313-11-19-abhab5.dcm)>, (1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313-11-20-hn94uu.dcm, 1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313-11-20-hn94uu.dcm): <FileData 1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313-11-20-hn94uu.dcm (1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313-11-20-hn94uu.dcm)>, (1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313-11-1-xb6jdm.dcm, 1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313-11-1-xb6jdm.dcm): <FileData 1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313-11-1-xb6jdm.dcm (1.3.6.1.4.1.14519.5.2.1.4429.7055.33

In [12]:
session.projects[projectLabel].subjects[subjectLabel].experiments[experimentNo].scans[scanNo].resources['DICOM'].files[0]

<FileData 1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313-11-19-abhab5.dcm (1.3.6.1.4.1.14519.5.2.1.4429.7055.331058148523327901159449546313-11-19-abhab5.dcm)>

In [13]:
fileObj = session.projects[projectLabel].subjects[subjectLabel].experiments[experimentNo].scans[scanNo].resources['DICOM'].files[0]

with fileObj.open() as fileObjOpen:
    fileObjOpen.read()
    
fileObjOpen

In [14]:
len(fileObjOpen)

TypeError: object of type 'RequestsFileLike' has no len()

In [15]:
fileObj = session.projects[projectLabel].subjects[subjectLabel].experiments[experimentNo].scans[scanNo].resources['DICOM'].files[0]

with fileObj.open() as fileObjOpen:
    temp = fileObjOpen.read(3000)
    
temp

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00DICM\x02\x00\x00\x00UL\x04\x00\xc4\x00\x00\x00\x02\x00\x01\x00OB\x00\x00\x02\x00\x00\x00\x00\x01\x02\x00\x02\x00UI\x1a\x001.2.840.10008.5.1.4.1.1.4\x00\x02\x00\x03\x00UI@\x001.3.6.1.4.1.14519.5.2.1.4429.7055.213859255991588180723164022910\x02\x00\x10\x00UI\x14\x001.2.840.10008.1.2.1\x00\x02\x00\x12\x00UI\x12\x001.2.40.0.13.1.1.1\x00\x02\x00\x13\x00SH\x0e\x00dcm4che-1.4.35\x08\x00\x05\x00CS\n\x00ISO_IR 100\x08\x00\x08\x00CS\x16\x00ORIGINAL\\PRIMARY\\OTHER\x08\x00\x16\x00UI\x1a\x001.

In [16]:
len(temp)

3000

Ran out of ideas for now..